# Imports

In [2]:
import os
import re
import nltk
import zipfile
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import precision_recall_fscore_support
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers  import Embedding
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras import layers
from keras import preprocessing
import pandas as pd

Using TensorFlow backend.


In [4]:
data=pd.read_pickle("data")

## Add new Features

F-measure 

In [5]:
adj=['JJ','JJR','JJS']
noun=['NN','NNS','NNPS','NNP']
verb=['VB','VBD','VBG','VBN','VBP','VBZ']
adverb=['RB','RBR','RBS']
article=['RP']
prep=['IN']
pronoun=['PRP','PRP$']
other=['CC','DT','CD','EX','FW','LS','MD','PDT','POS','SYM','TO','UH','WDT','WP','WP$','WRB','.',',']
dicttag={}
for i in adj:
    dicttag[i]='ADJ'
for i in noun:
    dicttag[i]='Noun'
for i in verb:
    dicttag[i]='Verb'
for i in adverb:
    dicttag[i]='ADVERB'
for i in pronoun:
    dicttag[i]='PRONOUN'
for i in other:
    dicttag[i]='OTHER'
dicttag['IN']='PREP'
dicttag['RP']='ART'

In [6]:
dicttagsent={}
dicttagsent['Noun']='n'
dicttagsent['Verb']='v'
dicttagsent['ADJ']='a'
dicttagsent['ADVERB']='r'
dicttagsent['PREP']='n'
dicttagsent['ART']='n'
dicttagsent['OTHER']='n'
dicttagsent['PRONOUN']='n'

In [7]:
def frequency(text):
    mytxt=nltk.word_tokenize(text)
    tag=nltk.pos_tag(mytxt)
    dictfreq={}
    for i in ['ADJ','Noun','Verb','ADVERB','PRONOUN','PREP','ART','OTHER']:
        dictfreq[i]=0
    for i in tag:
        if dicttag[i[1]] in dictfreq:
            dictfreq[dicttag[i[1]]]=dictfreq[dicttag[i[1]]]+1
        else:
            dictfreq[dicttag[i[1]]]=1
    return dictfreq
def F(text):
    dic=frequency(text)
    f=0.5*(dic['Noun']+dic['ADJ']+dic['PREP']+dic['ART']-dic['PRONOUN']-dic['Verb']-dic['ADVERB']+100)
    return f

Gender Preferential Features 

In [8]:
def f1(text):
    mytxt=nltk.word_tokenize(text)
    freq=0
    for i in mytxt:
        if i[-4:]=='able':
            freq=freq+1
    return freq
def f2(text):
    mytxt=nltk.word_tokenize(text)
    freq=0
    for i in mytxt:
        if i[-2:]=='al':
            freq=freq+1
    return freq
def f3(text):
    mytxt=nltk.word_tokenize(text)
    freq=0
    for i in mytxt:
        if i[-3:]=='ful':
            freq=freq+1
    return freq
def f4(text):
    mytxt=nltk.word_tokenize(text)
    freq=0
    for i in mytxt:
        if i[-4:]=='ible':
            freq=freq+1
    return freq
def f5(text):
    mytxt=nltk.word_tokenize(text)
    freq=0
    for i in mytxt:
        if i[-2:]=='ic':
            freq=freq+1
    return freq
def f6(text):
    mytxt=nltk.word_tokenize(text)
    freq=0
    for i in mytxt:
        if i[-3:]=='ive':
            freq=freq+1
    return freq
def f7(text):
    mytxt=nltk.word_tokenize(text)
    freq=0
    for i in mytxt:
        if i[-4:]=='less':
            freq=freq+1
    return freq
def f8(text):
    mytxt=nltk.word_tokenize(text)
    freq=0
    for i in mytxt:
        if i[-2:]=='ly':
            freq=freq+1
    return freq
def f9(text):
    mytxt=nltk.word_tokenize(text)
    freq=0
    for i in mytxt:
        if i[-3:]=='ous':
            freq=freq+1
    return freq

Surface features

In [9]:
from nltk.corpus import sentiwordnet as swn
def sentcount(text):
    return len(sent_tokenize(text))
def wordcount(text):
    return len(word_tokenize(text))
def characterscount(text):
    a=word_tokenize(text)
    l=0
    for i in a:
        l=l+len(i)
    return l
def alphabetcount(text):
    a=word_tokenize(text)
    l=0
    for i in a:
        for j in i:
            if j.isalpha():
                l=l+1
    return l
def digitscount(text):
    a=word_tokenize(text)
    l=0
    for i in a:
        for j in i:
            if j.isdigit():
                l=l+1
    return l
def specount(text):
    a=word_tokenize(text)
    l=0
    L=['@','#', '$', '%', '&', '*', '-', '=', '+', '¿', '¡', '[', ']', '{', '}', '/']
    for i in a:
        for j in i:
            if j in L:
                l=l+1
    return l
def punctcount(text):
    a=word_tokenize(text)
    l=0
    L=[',',';',':','!','?','.']
    for i in a:
        for j in i:
            if j in L:
                l=l+1
    return l
def shortcount(text):
    a=word_tokenize(text)
    l=0
    for i in a:
        if len(i)<5:
            l=l+1
    return l
def avgcount(text):
    a=word_tokenize(text)
    L=[len(i) for i in a]
    return sum(L)/len(L)
def sentiscore(text):
    pos=0
    neg=0
    l=0
    token = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(token)
    for i in tagged:
        tag=dicttagsent[dicttag[i[1]]]
        a=swn.senti_synsets(i[0], tag)
        posscore=0
        negscore=0
        for synst in a:
            posscore=posscore+synst.pos_score()
            negscore=negscore+synst.neg_score()
        if posscore>0 or negscore>0:
            pos=pos+posscore
            neg=neg+negscore
            l=l+1
    return pos/l,neg/l


In [11]:
import collections
import re

def tokenize(s):
    tokens = re.split(r"[^0-9A-Za-z\-'_]+", s)
    return tokens

def get_yules(s):
    
    tokens = tokenize(s)
    token_counter = collections.Counter(tok.upper() for tok in tokens)
    m1 = sum(token_counter.values())
    m2 = sum([freq ** 2 for freq in token_counter.values()])
    i = (m1*m1) / (m2-m1)
    k = 1/i * 10000
    return k

POS Sequence Pattern Features

In [12]:
femclass=['shopping','mom','cried','freaked','pink','cute','gosh','kisses','yummy','mommy','boyfriend','skirt','adorable','husband','hubby']
malclass=['linux',  'microsoft',  'gaming',  'server',  'software',  'gb', 'programming',  'google',  'data',  'graphics',  'india',  'nations', 'democracy', 'users', 'economic']
acroclass=['gm', 'gn',  'brb',  'rofl','afaik',  'lmao', 'lmfao', 'lol',  'tgif', 'omg', 'omfg', 'tc', 'ttyl', 'af', 'ama', 'bae', 'dm', 'dafuq', 'eli5', 'fml', 'ftfy', 'hifw', 'icymi', 'imho', 'irl', 'mrw', 'yolo', 'xoxo']
chastclass=['sexual', 'celibacy', 'abstinence',  'virtue',  'monogamy',  'virginity', 'pureness',  'unchaste',  'fasting',  'frugality',  'self-restraint', 'abnegation',  'asceticism',  'avoidance',  'renunciation',  'self-control',  'sobriety',  'teetotalism',  'monasticism',  'austerity', 'abstain', 'diet', 'hungry ']
predclass=['forecast', 'guess', 'prognosis',  'prophecy',  'augury',  'conjecture', 'divination',  'foretell',  'hunch',  'horoscope',  'omen',  'presage', 'prevision', 'zodiac', 'fortune-telling', 'surmising']
envyclass=['hatred',  'malice',  'prejudice',  'rivalry',  'backbiting',  'coveting', 'grudge',  'heartburn', 'malevolence',  'opposition', 'green-eyed', 'invidious', 'resentful']
rainclass=['deluge',  'drizzle',  'flood', 'hail',  'mist', 'monsoon',  'precipitation', 'rainfall',  'shower',  'sleet', 'stream',  'torrent',  'pour', 'spate',  'spit', 'sprinkle', 'wet-stuff']
evilclass=['criminal',  'devil',  'felon',  'gangster',  'lawbreaker',  'murderer', 'psychopath', 'sinner', 'sociopath', 'troublemaker', 'villain']
mathclass=['mathematics', 'algebra', 'arithmetic', 'geometry', 'trigonometry', 'calculus',  'math', 'addition',  'division', 'multiply',  'figures',  'pie', 'chart', 'numbers', 'subtraction', 'plot', 'graph', 'data']
groomclass=['groom', 'bride', 'suitor', 'benedict', 'spouse', 'fiancé']
ethiclass=['ethics', 'belief', 'conduct', 'convention', 'honesty', 'honor', 'mores', 'values', 'ethos', 'practice', 'principles', 'conscience']
fearclass=['fainthearted',  'terrified',  'angst',  'anxiety',  'concern',  'despair', 'dread',  'horror',  'jitters',  'panic',  'unease',  'worry',  'agitation', 'aversion',  'awe',  'consternation',  'creeps',  'discomposure', 'disquietude', 'distress', 'fright', 'nightmare', 'phobia']
disgclass=['antipathy', 'dislike',  'distaste',  'loathe', 'revulsion', 'abhorrence', 'abominate', 'detest', 'hateful', 'nausea', 'objection', 'repugnance', 'revolt', 'satiation', 'satiety', 'sickness', 'surfeit']

In [13]:
def freqclass(text):
    dict={}
    classes=['femclass','malclass','acroclass','chastclass','predclass','envyclass','rainclass','evilclass','mathclass','groomclass','ethiclass','fearclass','disgclass']
    for i in classes:
        dict[i]=0
    a=word_tokenize(text)
    for i in a:
        if i in femclass:
            dict['femclass']=dict['femclass']+1
        elif i in malclass:
            dict['malclass']=dict['malclass']+1
        elif i in acroclass:
            dict['acroclass']=dict['acroclass']+1
        elif i in chastclass:
            dict['chastclass']=dict['chastclass']+1
        elif i in groomclass:
            dict['groomclass']=dict['groomclass']+1
        elif i in ethiclass:
            dict['ethiclass']=dict['ethiclass']+1
        elif i in rainclass:
            dict['rainclass']=dict['rainclass']+1
        elif i in evilclass:
            dict['evilclass']=dict['evilclass']+1
        elif i in envyclass:
            dict['envyclass']=dict['envyclass']+1
        elif i in fearclass:
            dict['fearclass']=dict['fearclass']+1
        elif i in disgclass:
            dict['disgclass']=dict['disgclass']+1
        elif i in predclass:
            dict['predclass']=dict['predclass']+1
        elif i in mathclass:
            dict['mathclass']=dict['mathclass']+1
    return dict
    

## Train with new features

In [14]:
Data = pd.read_pickle("Dataf1")

## ModelTrain

In [15]:
from sklearn.model_selection import train_test_split
#Data=Data.drop('post',axis=1)
y = data.gender.values
X_train, X_test, y_train, y_test = train_test_split(Data, y, test_size=0.2)

In [16]:
Y_train=np.zeros(y_train.shape)
for i in range(len(y_train)):
    if y_train[i]=='male':
        Y_train[i]=1

In [17]:
Y_test=np.zeros(y_test.shape)
for i in range(len(y_test)):
    if y_test[i]=='male':
        Y_test[i]=1
X_test['age']=X_test['age'].astype(float)

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [18]:
X_train['age']=X_train['age'].astype(float)
X_test['age']=X_test['age'].astype(float)

/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/infres/tabarani/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
from xgboost import XGBClassifier
xgb=XGBClassifier(n_jobs=20)
xgb.fit(X_train,Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=20, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

## ModelTest

In [22]:
y_pred=xgb.predict(X_test)

In [23]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,Y_test)

0.604236081551175